# Custom agents with custom search

In [7]:
from dotenv import load_dotenv

from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate

from langchain import OpenAI, LLMChain
from langchain.tools import DuckDuckGoSearchRun 

from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re
import langchain

load_dotenv()

True

## Setup Tools

In [2]:
# Define which tools the agent can use to answer user queries
search = DuckDuckGoSearchRun()

tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

In [3]:
obj = search.run("How can I treat a sprained ankle?")
type(obj)

str

In [4]:
obj

'Learn how to recognize and treat a sprained ankle, a common injury that happens when a ligament is stretched or torn. Find out the risk factors, recovery time, and when to see a doctor for a ... Learn how to treat and prevent ankle sprains, common musculoskeletal injuries that cause pain, swelling, and difficulty walking. Find out the grades of severity, the RICE regimen, and the exercises to restore function and balance. Stirrup brace: This type of brace has air cells in a plastic frame that run along either side of your ankle, with a "stirrup" under your heel. This brace provides a high level of support—it allows you to walk, but keeps your ankle from rolling. Lace-up ankle braces: These ankle braces slide over your foot and ankle and are "cinched" with laces, like a shoe. To treat a sprain, try the R.I.C.E. approach — rest, ice, compression, elevation: Rest the injured area. Your healthcare professional may say not to put weight on the injured area for 48 to 72 hours. You may need 

In [5]:
search.run("site:webmd.com How can I treat a sprained ankle?")

"Sprained ankle vs. broken ankle. While a sprained ankle involves torn or stretched ligaments, a broken ankle means that at least one of the bones in your ankle is broken. Symptoms of a sprain and ... The RICE method is a simple self-care technique that aims to reduce swelling, ease pain, and speed up healing. Rest, ice, compression, and elevation of a sprain or strain are part of the RICE ... The most common type of sprain is a sprained ankle. About 25,000 people sprain an ankle every day. ... Most people with mild sprains and strains can treat these injuries at home by following ... A toe sprain may happen when you jam, stub, or overextend any of your toes, damaging the tendon or soft tissues of the toe. If you don't have a fracture, the pain and swelling should go away ... They can get inflamed from arthritis, overuse, high-heeled shoes, recent footwear changes, or starting workouts again after time off. Your ankle may feel stiff, tender, warm, and swollen. The best ..."

In [6]:
def duck_wrapper(input_text):
    search_results = search.run(f"site:webmd.com {input_text}")
    return search_results

tools = [
    Tool(
        name = "Search WebMD",
        func=duck_wrapper,
        description="useful for when you need to answer medical and pharmalogical questions"
    )
]

## Prompt template

This instructs the agent on what to do. Generally, the template should incorporate:

**`tools:`** which tools the agent has access and how and when to call them.

**`intermediate_steps:`** These are tuples of previous (**`AgentAction`**, **`Observation`**) pairs. These are generally not passed directly to the model, but the prompt template formats them in a specific way.


In [8]:
# Set up the base template
template = """Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to answer as a compansionate medical professional when giving your final answer.

Question: {input}
{agent_scratchpad}"""

In [9]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [10]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

## Custom Output Parser

The output parser is responsible for parsing the LLM output into AgentAction and AgentFinish. This usually depends heavily on the prompt used.

This is where you can change the parsing to do retries, handle whitespace, etc

In [11]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [12]:
output_parser = CustomOutputParser()

## Set up LLM

In [13]:
llm = OpenAI(temperature=0)

/var/folders/tf/41xzfdmn66gc1qv5sv001p_w0000gp/T/ipykernel_69368/3183534344.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0)
